In [26]:
import torch
import pandas as pd
import torch.nn as nn
from torchvision import transforms as T
from torchvision.transforms import InterpolationMode
from torchviz import make_dot
import numpy as np
import os
from PIL import Image


import default as c
from utils import Score_Observer, load_weights, save_weights
from models.extractors import build_extractor
from models.flow_models import build_msflow_model
from datasets import CKMDataset
from post_process import post_process
from evaluations import eval_det_loc_only
from train import train_meta_epoch, inference_meta_epoch

In [8]:
weight_path = 'E:\\codes\\ckm\\modeling\\MSFlow\\work_dirs\\msflow_wide_resnet50_2_avgpool_pl258\\textile\\best_det.pt'

In [10]:

extractor, output_channels = build_extractor(c)
extractor = extractor.to(c.device).eval()
parallel_flows, fusion_flow = build_msflow_model(c, output_channels)  # MSFlow 모델 생성
parallel_flows = [parallel_flow.to(c.device) for parallel_flow in parallel_flows]
fusion_flow = fusion_flow.to(c.device)


_ = load_weights(parallel_flows, fusion_flow, weight_path)

Channels of extracted features: [256, 512, 1024]
Build parallel flows: channels:256, block:2, cond:64
Build parallel flows: channels:512, block:5, cond:64
Build parallel flows: channels:1024, block:8, cond:64
Build fusion flow with channels [256, 512, 1024]
Loading weights from E:\codes\ckm\modeling\MSFlow\work_dirs\msflow_wide_resnet50_2_avgpool_pl258\textile\best_det.pt


In [30]:
data = {'name':[],
        'weight':[]}


for name, param in extractor.named_parameters():
    if param.requires_grad:
        print(name,'->', param.data.abs().mean())
        data['name'].append(name)
        data['weight'].append(param.data.abs().mean().cpu().numpy())

conv1.weight -> tensor(0.0705, device='cuda:0')
bn1.weight -> tensor(0.2466, device='cuda:0')
bn1.bias -> tensor(0.2521, device='cuda:0')
layer1.0.conv1.weight -> tensor(0.0288, device='cuda:0')
layer1.0.bn1.weight -> tensor(0.1395, device='cuda:0')
layer1.0.bn1.bias -> tensor(0.0817, device='cuda:0')
layer1.0.conv2.weight -> tensor(0.0085, device='cuda:0')
layer1.0.bn2.weight -> tensor(0.1161, device='cuda:0')
layer1.0.bn2.bias -> tensor(0.1025, device='cuda:0')
layer1.0.conv3.weight -> tensor(0.0134, device='cuda:0')
layer1.0.bn3.weight -> tensor(0.1279, device='cuda:0')
layer1.0.bn3.bias -> tensor(0.0508, device='cuda:0')
layer1.0.downsample.0.weight -> tensor(0.0264, device='cuda:0')
layer1.0.downsample.1.weight -> tensor(0.1790, device='cuda:0')
layer1.0.downsample.1.bias -> tensor(0.0508, device='cuda:0')
layer1.1.conv1.weight -> tensor(0.0137, device='cuda:0')
layer1.1.bn1.weight -> tensor(0.1319, device='cuda:0')
layer1.1.bn1.bias -> tensor(0.0905, device='cuda:0')
layer1.1.con

In [31]:
df = pd.DataFrame(data=data, columns=['name', 'weight'])
df

,name,weight
0,conv1.weight,0.07052137
1,bn1.weight,0.24656987
2,bn1.bias,0.25208694
3,layer1.0.conv1.weight,0.02882829
4,layer1.0.bn1.weight,0.13946417
...,...,...
124,layer3.5.bn2.weight,0.13051416
125,layer3.5.bn2.bias,0.09781123
126,layer3.5.conv3.weight,0.008329967
127,layer3.5.bn3.weight,0.07343365


In [32]:
df.sort_values(by='weight', ascending=False).head(10)

,name,weight
2,bn1.bias,0.25208694
1,bn1.weight,0.24656987
13,layer1.0.downsample.1.weight,0.179041
73,layer3.0.bn1.weight,0.17038327
37,layer2.0.bn2.weight,0.16845101
28,layer1.2.bn2.weight,0.15533565
34,layer2.0.bn1.weight,0.15139613
25,layer1.2.bn1.weight,0.14925739
58,layer2.2.bn2.weight,0.14684612
76,layer3.0.bn2.weight,0.14608306


In [17]:
x = Image.open('E:\\codes\\ckm\\modeling\\MSFlow\\data\\CKM\\test\\bad\\1_3.png').convert('RGB')

transform_x = T.Compose([
        T.Resize(c.input_size, InterpolationMode.LANCZOS),
        T.ToTensor()])
normalize = T.Compose([T.Normalize(c.img_mean, c.img_std)])

x = normalize(transform_x(x))
y = 1

In [22]:
x = x.to(c.device).unsqueeze(0)
h_list = extractor(x)

for h in h_list:
    make_dot(h).view()

torch.Size([1, 1, 3, 128, 128])


NotImplementedError: Only 2D, 3D, 4D, 5D padding with non-constant padding are supported for now